### Global Carbon Budget

Plot GCB2023, with data through end 2022
Consider impacts on the atmosphere if the ocean and land sinks had been of different magnitudes

Designed for Humans and the Carbon Cycle, Fall 2024 (EESC GU 4020) at Columbia University \
August 7 2024 draft1 1, A. Shaum and Professor G.A. McKinley \
September 7 2024 for Problem Set 1

In [ ]:
# This file holds the plotting functions. It is required. 
%run plotting_functions.ipynb

In [ ]:
# Loading packages
import pandas as pd
import openpyxl # <- needed to open xlsx
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
%matplotlib widget 
# ^ allows zooming in to the plots!

import cmocean as cm    

import gcsfs
fs = gcsfs.GCSFileSystem()

### Load data from GCB2023, excel file released with paper

In [ ]:
# Data from the Global Carbon Budget, HistoricalBudget page 
# from file Global_Carbon_Budget_2023v1.0.xlsx at https://essd.copernicus.org/articles/15/5301/2023/
GCB23historical = pd.read_excel('data/Global_Carbon_Budget_2023v1.1.xlsx',sheet_name='Historical Budget',skiprows=15,nrows=273)

### extract 1850 forward; stack plot as in Figure 3 of GCB

Use historical file since the timeseries are already connected between the pre-modern and modern estimates

In [ ]:
GCB23historical1850 = GCB23historical[GCB23historical['Year'].between(1850,2023)]

In [ ]:
GCB23historical1850

### Stacked plot has emissions (positive) and sinks (negative). Sinks include atmosphere. Each source / sink is progressively added. 

This is a closed system, so the sum of sources must equal the sum of sinks. The red dash line mirrors the total emissions. If we knew the budgets perfectly, the red dash line would fall on the atmosphere exactly. The lack of concurrence indicates our inability to precisely close the budget.

In [ ]:
# getting total time range
time = GCB23historical1850['Year']

# calculating emissions
fossil_fuels = GCB23historical1850['fossil emissions excluding carbonation']
land_use = GCB23historical1850['land-use change emissions']

total_emissions = fossil_fuels + land_use

# calculating sinks: made negative to emphasize that they are SINKS!
ocean_sink = -GCB23historical1850['ocean sink']
land_sink = -GCB23historical1850['land sink']
atmo_sink = -GCB23historical1850['atmospheric growth']

### 1. Plot GCB2023
Use the functions in top left to zoom in\
Click on floppy disk icon to save PNG to desktop

In [ ]:
fig = GCB_plot(time, total_emissions,land_use,ocean_sink,land_sink,oceanfactor=1,landfactor=1)

## 1a. Add a factor to ocean or land sink and see what would have happened to atmosphere if this sink had been smaller

Define a value between 0 and 1 to scale the estimated ocean and land sinks

In [ ]:
# Run this cell to set the factors you want to use
ocean_scale = 0.5
land_scale = 0.5

In [ ]:
fig = GCB_plot(time, total_emissions,land_use,ocean_sink,land_sink,oceanfactor=ocean_scale,landfactor=land_scale)

## 1b. Direct comparison of actual budget to modified

In [ ]:
fig = GCBcompare_scale(time, total_emissions,land_use,ocean_sink,land_sink,oceanfactor=ocean_scale,landfactor=land_scale)

## 2. Bar plot comparing cumulative emissions to cumulative sinks from 1850 to 2022
Click on floppy disk icon to save PNG to desktop

In [ ]:
bar_plot(fossil_fuels,land_use,land_sink,ocean_sink,atmo_sink)

## 2a. Apply scalings to land and ocean sinks and plot

In [ ]:
bar_plot(fossil_fuels,land_use,land_sink,ocean_sink,atmo_sink,oceanfactor=ocean_scale,landfactor=land_scale)

## 3. Sum the atmosphere from 1959 onward and compare to Mauna Loa CO2 observations 

Then will apply factor for "what if ocean were stronger or weaker" and can see impact to atmosphere

In [ ]:
# data source for CO2 mole fraction (ppm) measured monthly at Mauna Loa since 1958
# https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.csv

# The atmosphere is rapidly mixed by the winds, so this timeseries is a good proxy for the global mean atmospheric pCO2
xCO2atm_ML = pd.read_csv('data/co2_mm_mlo.csv', skiprows=40,nrows=796) 

In [ ]:
# Since the mauna loa starts in mid-1958, we restrict the data to starting 1959
GCB23historical1959 = GCB23historical[GCB23historical['Year'].between(1959,2023)]

### Compare the atmospheric record in the Global Carbon Budget to that observed at Mauna Loa, Hawai'i

This plotting function used here \
(1) sums the atmospheric growth from GCB to get cumulative GtC \
(2) converts to ppm using 2.124GtC = 1 ppm \
(3) adds to the 1958 value of 315ppm 

See plotting_functions.ipynb for detail

### Plot Mauna Loa vs. GCB
Use the functions in top left to zoom in \
Click on floppy disk icon to save PNG to desktop

In [ ]:
atmospheric_co2_plot(GCB23historical1959, xCO2atm_ML)

The Mauna Loa monthly data and our integration of the GCB annual growth rate give the same estimate of global atmospheric CO2 

## 3a. Apply scalings to land and ocean sinks and plot. How would atmospheric pCO2 have been different? 
Use the functions in top left to zoom in

In [ ]:
atmospheric_co2_plot(GCB23historical1959, xCO2atm_ML, oceanfactor=ocean_scale, landfactor=land_scale)